In [2]:
import matplotlib.pyplot as plt
import numpy as np
from custom_sepsis import *

/Users/luisastue/miniconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
import json
import numpy as np

# Define parameter sets
param_sets = {
    "small_buff": {
        "buffer_size": 50000,
        "batch_size": 16,
        "train_freq": 1,
        "gradient_steps": 1,
        "exploration_fraction": 0.2,
        "exploration_final_eps": 0.05,
        "target_update_interval": 1000,
    },
    "large_buff": {
        "buffer_size": 1000000,
        "batch_size": 256,
        "train_freq": 8,
        "gradient_steps": 20,
        "exploration_fraction": 0.05,
        "exploration_final_eps": 0.01,
        "target_update_interval": 1000,
    },
    "medium": {
        "buffer_size": 500000,
        "batch_size": 128,
        "train_freq": 4,
        "gradient_steps": 10,
        "exploration_fraction": 0.2,
        "exploration_final_eps": 0.05,
        "target_update_interval": 500,
    },
    "small_buff_small_expl": {
        "buffer_size": 50000,
        "batch_size": 16,
        "train_freq": 1,
        "gradient_steps": 10,
        "exploration_fraction": 0.05,
        "exploration_final_eps": 0.01,
        "target_update_interval": 1000,
    },
}

# Configuration
n_runs = 30
n_steps = 200000
metric = "small_buff_small_expl"

tasks = [
    (metric, param_sets[metric], run, n_steps)
    for run in range(2, 23) 
]

for task in tasks:
    train_and_log_rewards(task)
